# IFT870 - Forage de données - TP2

Membres de l'équipe :
- Joë Larue
- Aurélien Vauthier

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# setup notebook parameters
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# extract data
package = pd.read_csv("Package2.csv", delimiter=';', encoding="latin1")
product = pd.read_csv("Product2.csv", delimiter=';', encoding="latin1")

## 1. Auscultez les données et présentez un résumé de votre auscultation

Une première observation des données grâce à microsoft Excel, ainsi que des descriptions en ligne des fichiers 
[product](https://www.fda.gov/drugs/drug-approvals-and-databases/ndc-product-file-definitions) et
[package](https://www.fda.gov/drugs/drug-approvals-and-databases/ndc-package-file-definitions) nous a permis d'avoir un
premier aperçu du contenu de nos tables et leurs significations.

In [3]:
print(f"Affichons un aperçu des premières observations de package (de taille : {package.shape}) :")
package.head()

Affichons un aperçu des premières observations de package (de taille : (173887, 8)) :


,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N


In [4]:
print(f"Affichons un aperçu des premières observations de product (de taille : {product.shape}) :")
product.head()

Affichons un aperçu des premières observations de product (de taille : (93238, 20)) :


,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0


Notre première observation est qu'il existe plus de packages que de produits, ce qui nous laisse penser qu'un produit
 sera probablement présent dans plusieurs packages.

## 1.1 Correctitude des données

In [36]:
product.dtypes

PRODUCTID                                   object
PRODUCTNDC                                  object
PRODUCTTYPENAME                             object
PROPRIETARYNAME                             object
PROPRIETARYNAMESUFFIX                       object
NONPROPRIETARYNAME                          object
DOSAGEFORMNAME                              object
ROUTENAME                                   object
STARTMARKETINGDATE                  datetime64[ns]
ENDMARKETINGDATE                    datetime64[ns]
MARKETINGCATEGORYNAME                       object
APPLICATIONNUMBER                           object
LABELERNAME                                 object
SUBSTANCENAME                               object
ACTIVE_NUMERATOR_STRENGTH                   object
ACTIVE_INGRED_UNIT                          object
PHARM_CLASSES                               object
DEASCHEDULE                                 object
NDC_EXCLUDE_FLAG                            object
LISTING_RECORD_CERTIFIED_THROUG

In [14]:
package.dtypes

PRODUCTID                     object
PRODUCTNDC                    object
NDCPACKAGECODE                object
PACKAGEDESCRIPTION            object
STARTMARKETINGDATE    datetime64[ns]
ENDMARKETINGDATE      datetime64[ns]
NDC_EXCLUDE_FLAG              object
SAMPLE_PACKAGE                object
dtype: object

Le type objet signifie qu'au sein d'une même colonne, il y a un mélange de plusieurs types. En particulier cela arrive
lorsque des valeurs manquantes (représenté par `np.NaN`) sont présentent.  
Certains attributs représentent des dates et n'ont pas le format idéal comme:
- `STARTMARKETINGDATE`
- `ENDMARKETINGDATE`
- `LISTING_RECORD_CERTIFIED_THROUGH`

Changeont donc leurs types :

In [3]:
# set product columns (use coerce error to eliminate outliers, like date set around the year 3000)
product["STARTMARKETINGDATE"] = pd.to_datetime(product["STARTMARKETINGDATE"], format="%Y%m%d")
product["ENDMARKETINGDATE"] = pd.to_datetime(product["ENDMARKETINGDATE"], format="%Y%m%d", errors="coerce")
product["LISTING_RECORD_CERTIFIED_THROUGH"] = pd.to_datetime(product["LISTING_RECORD_CERTIFIED_THROUGH"], format="%Y%m%d")

# set package columns (use coerce error to eliminate outliers, like date set around the year 3000)
package["STARTMARKETINGDATE"] = pd.to_datetime(package["STARTMARKETINGDATE"], format="%Y%m%d", errors="coerce")
package["ENDMARKETINGDATE"] = pd.to_datetime(package["ENDMARKETINGDATE"], format="%Y%m%d", errors="coerce")

print("Toutes les colonnes ont bien été changé pour le type date.")

Toutes les colonnes ont bien été changé pour le type date.


## 1.2 Complétude

In [6]:
product.isna().sum()

PRODUCTID                            1560
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
PROPRIETARYNAME                         6
PROPRIETARYNAMESUFFIX               83075
NONPROPRIETARYNAME                      4
DOSAGEFORMNAME                          0
ROUTENAME                            1932
STARTMARKETINGDATE                     20
ENDMARKETINGDATE                    88916
MARKETINGCATEGORYNAME                   0
APPLICATIONNUMBER                   13097
LABELERNAME                             0
SUBSTANCENAME                        2309
ACTIVE_NUMERATOR_STRENGTH            2309
ACTIVE_INGRED_UNIT                   2309
PHARM_CLASSES                       50984
DEASCHEDULE                         88815
NDC_EXCLUDE_FLAG                        0
LISTING_RECORD_CERTIFIED_THROUGH     4325
dtype: int64

In [7]:
package.isna().sum()

PRODUCTID                  0
PRODUCTNDC              1500
NDCPACKAGECODE          2346
PACKAGEDESCRIPTION         0
STARTMARKETINGDATE       269
ENDMARKETINGDATE      167431
NDC_EXCLUDE_FLAG           0
SAMPLE_PACKAGE             0
dtype: int64

De nombreuses valeurs sont manquantes, cependant il est probable que certaines valeurs soit récupérables en croisant
 les colonnes comme pour le PRODUCTNDC par exemple avec les colonnes `PRODUCTID` et `PACKAGEDESCRIPTION`.

## 1.3 Cohérence

Le `PRODUCTNDC` est présent dans la construction de plusieurs autres attributs :  
- `PRODUCTID`
- `NDCPACKAGECODE`

L'`APPLICATIONNUMBER` est lié à la valeur de `MARKETINGCATEGORYNAME`.

Le `PACKAGEDESCRIPTION` a un lien avec :  
- `SUBSTANCENAME`
- `ACTIVE_NUMERATOR_STRENGTH`
- `ACTIVE_INGRED_UNIT`

De plus, ces trois colonnes forme un trio : pour chaque substance dans `SUBSTANCENAME` ont retrouve une valeur de concentration dans
le médicamment dans `ACTIVE_NUMERATOR_STRENGTH` et l'unité de la concentration dans `ACTIVE_INGRED_UNIT`

`LISTING_RECORD_CERTIFIED_THROUGH` a un lien avec `NDC_EXCLUDE_FLAG`.

Nous expliquerons ces liens dans la partie `1.5`.

## 1.4 Données à jour

`LISTING_RECORD_CERTIFIED_THROUGH` représente la date de fin de validité des informations sur un médicament.
Beaucoup de valeurs sont manquantes, cependant la présence de `N` et non pas `E` dans la colonne de `NDC_EXCLUDE_FLAG`
nous indique que les informations sont supposées être à jour.

On peut faire 2 hypothèses, les lignes où `LISTING_RECORD_CERTIFIED_THROUGH` est manquant ne sont pas à jour, et
donc, la ligne `NDC_EXCLUDE_FLAG` est fausse et on ne traite pas ces lignes. Ou alors on se fie à la ligne
`NDC_EXCLUDE_FLAG` est on les traites quand même.

Nous allons suivre la 2e hypothèse et continuer à utiliser les données en question.

## 1.5 Données vraisemblables et interprétables

Nous allons passer au cas par cas les attributs et identifier le maximum de choses à dire à leur sujet.

### Attributs présent dans les 2 tables:

#### PRODUCTID
Comme identifié au-dessus, il y a des valeurs manquantes dans la tables product uniquement.  
Il y a aussi des éléments en double dans la table des packages, ce qui a du sens à priori car plusieurs médicaments
 peuvent être présent dans différents packagings.
Le productid est la fusion entre product NDC (connue) et SPL document ID (inconnue).
Ces colonnes ne possèdent pas d'information pertinente en dehors de leur utilité pour fusionner les 2 tables.

#### PRODUCTNDC  
Cet attribut est la concaténation du code de l'étiqueteur.se et du code du produit. Ces 2 éléments vont de pair pour
identifier un médicament.
Les différents formats sont 4-4, 5-3, ou 5-4 (ex: 12345-1234).
Des valeurs sont manquantes dans la table package, il se peut qu'une récupération puisse être faite grâce au `PRODUCTID`
ou au `PACKAGEDESCRIPTION`.
En parcourant le fichier `product`, nous avons trouvé des données aberrantes comme des dates par exemple.

#### STARTMARKETINGDATE

In [3]:
print(f"Valeur Max dans Product: {product['STARTMARKETINGDATE'].max()}")
print(f"Valeur Min dans Product: {product['STARTMARKETINGDATE'].min()}")
print(f"Valeur Max dans Package: {package['STARTMARKETINGDATE'].max()}")
print(f"Valeur Min dans Package: {package['STARTMARKETINGDATE'].min()}")

Valeur Max dans Product: 2020-02-14 00:00:00
Valeur Min dans Product: 1900-01-01 00:00:00
Valeur Max dans Package: 2020-02-14 00:00:00
Valeur Min dans Package: 1900-01-01 00:00:00


  
En parcourant la colonne, nous avons identifié la présence de bruits.  

Par exemple, la date 2997-12-24 doit probablement correspondre à 1997-12-24 avec une faute de frappe.  

Pour ce qui est de 1900-01-01, cela doit probablement être dû au fait que la donnée est inconnue.  

In [4]:
product["STARTMARKETINGDATE"].mask(product["STARTMARKETINGDATE"] == "1900-01-01", inplace=True)
package["STARTMARKETINGDATE"].mask(package["STARTMARKETINGDATE"] == "1900-01-01", inplace=True)
print("Les dates par défaut définie au 1900-01-01 ont été retirée.")

Les dates par défaut définie au 1900-01-01 ont été retirée.


#### ENDMARKETINGDATE  

En parcourant la colonne, nous avons identifié la présence de bruits ou de données aberrantes que nous corrigeons plus
haut lors de la conersion des colonnes en datetime.
Cependant, le très grand nombre de valeurs manquantes dans cette colonne réduit considérablement l'importance que
nous allons lui attribuer.

In [13]:
print(f"Valeur Max dans Product: {product['ENDMARKETINGDATE'].max()}")
print(f"Valeur Min dans Product: {product['ENDMARKETINGDATE'].min()}")
print(f"Valeur Max dans Package: {package['ENDMARKETINGDATE'].max()}")
print(f"Valeur Min dans Package: {package['ENDMARKETINGDATE'].min()}")

Valeur Max dans Product: 2039-08-31 00:00:00
Valeur Min dans Product: 2020-02-15 00:00:00
Valeur Max dans Package: 2039-08-31 00:00:00
Valeur Min dans Package: 2020-02-15 00:00:00


#### NDC_EXCLUDE_FLAG

D'après la description des 2 tables, il semble normal que l'on n'ait que des `N` dans cette colonne.
Les objets ayant des valeurs différentes ont, en effet, été mis dans des fichiers à part que nous ne possédons pas.

In [ ]:
print(f"Nombre de valeur unique dans package: {package['NDC_EXCLUDE_FLAG'].nunique()}")
print(f"Nombre de valeur unique dans Product: {product['NDC_EXCLUDE_FLAG'].nunique()}")

### Attributs présent uniquement dans la table Product:

#### PRODUCTTYPENAME

In [16]:
product["PRODUCTTYPENAME"].unique()

array(['HUMAN OTC DRUG', 'HUMAN PRESCRIPTION DRUG', 'VACCINE',
       'PLASMA DERIVATIVE', 'CELLULAR THERAPY',
       'NON-STANDARDIZED ALLERGENIC', 'STANDARDIZED ALLERGENIC'],
      dtype=object)

Il n'y a pas de données manquantes, et les valeurs semblent correctes.

#### PROPRIETARYNAME

In [21]:
product["PROPRIETARYNAME"][93066:93072]

93066             Arnica 12c
93067         ARNICA MONTANA
93068             ARNICA 30C
93069                 ARNICA
93070              Arnica 6c
93071    Arsenicum Album 30c
Name: PROPRIETARYNAME, dtype: object

Cet attribut décrit le nom du médicament.
Beaucoup de valeurs différentes sont présentes mais aussi des valeurs qui se ressemblent.
Il est difficile d'identifier comment nettoyer les données autrement qu'au cas par cas.

#### PROPRIETARYNAMESUFFIX

Cet attribut est optionnel, ce qui explique le très grand nombre de cases vide.
Le manque de standard de rédaction réduit l'intérêt à porter sur cet attribut.

#### NONPROPRIETARYNAME

Les valeurs écrites dans cet attribut peuvent être sous forme de liste. Cependant les listes peuvent utiliser des
séparateurs tels que ',' ou 'and' et certains mots sont présents en majuscules / minuscules.

De plus, plusieurs valeurs sont manquantes.

In [22]:
product["NONPROPRIETARYNAME"][543]

'Butalbital, Aspirin, Caffeine, and Codeine Phosphate'

#### DOSAGEFORMNAME

Cet attribut décrit l'apparence du médicament, les valeurs semblent correctes et il n'y a pas de valeur est manquante.

#### ROUTENAME

Cet attribut définit les voies d'administration du médicament par le patient.
Plusieurs voies peuvent être utilisées, et ces valeurs sont séparées par des ';'.
Plusieurs valeurs sont manquantes.

#### MARKETINGCATEGORYNAME

Cet attribut semble correct car il n'y a pas de valeur manquante.  
Cet attribut possède un lien étroit avec l'attribut suivant.

#### APPLICATIONNUMBER

Cet attribut dépend du précédent. On y retrouve les codes NDA/ANDA/BLA, sinon "partX" pour les catégories OTC
ou l'absence de valeur pour les "UNAPPROVED".

#### LABELERNAME

Certaines valeurs sont aberrantes, d'autres valides mais similaires.

In [25]:
product["LABELERNAME"][25785:25790]

25785                MEIJER, INC.
25786     Meijer Distribution Inc
25787    MEIJER DISTRIBUTION, INC
25788                      Meijer
25789    Meijer Distribution Inc.
Name: LABELERNAME, dtype: object

#### SUBSTANCENAME

Cet attribut représente la liste de substances dans le médicament, le caractère `;` est utilisé comme séparateur.
Beaucoup de valeurs sont manquantes. Enfin, nous pouvons aussi noter comme l'indique le [site de la FDA](https://www.fda.gov/drugs/drug-approvals-and-databases/ndc-product-file-definitions)
qu'il s'agit de l'attribut utilisé pour définir la classe pharmaceutic.

#### ACTIVE_NUMERATOR_STRENGTH

Cet attribut représente la quantité des substances décrites dans SUBSTANCENAME dans le médicament.
La valeur est manquante quand SUBSTANCENAME est manquant.

#### ACTIVE_INGRED_UNIT

Cet attribut représente l'unité des substances décrites dans SUBSTANCENAME dans le médicament.
La valeur est manquante quand SUBSTANCENAME est manquant.

#### PHARM_CLASSES

Cet attribut représente les classes pharmaceutiques auxquelles appartient le médicament.
Il peut y en avoir plusieurs par médicament, séparée par des `,`.

Beaucoup de données sont manquantes.

#### DEASCHEDULE

In [37]:
product["DEASCHEDULE"].unique()

array([nan, 'CV', 'CIV', 'CIII', 'CII'], dtype=object)

Cet attribut peut prendre les valeurs de {CI, CII, CIII, CIV, CV}.  
Beaucoup de valeurs sont manquantes, et il n'y a aucun CI.  
Ces valeurs représentent le ratio entre les facultés médicales d'une substance et la possibilité d'en abuser ou d'en être dépendant.  
CI représente les drogues telles que l'héroïne ou le LSD, c'est pourquoi CI n'est présent nul par dans cette table.  
Cet attribut dépend donc de SUBSTANCENAME.
Notre hypothèse est que les valeurs manquantes correspondent à des substances à dépendance nulle.

In [43]:
product.iloc[0][["SUBSTANCENAME", "DEASCHEDULE"]]

SUBSTANCENAME    WATER
DEASCHEDULE        NaN
Name: 0, dtype: object

#### LISTING_RECORD_CERTIFIED_THROUGH

Cet attribut représente la fin de validité de l'information contenue dans une ligne.
Certaines données sont manquantes, les autres semblent correctes.

In [46]:
print(f"Valeur Max pour LISTING_RECORD_CERTIFIED_THROUGH: {product['LISTING_RECORD_CERTIFIED_THROUGH'].max()}")
print(f"Valeur Min pour LISTING_RECORD_CERTIFIED_THROUGH: {product['LISTING_RECORD_CERTIFIED_THROUGH'].min()}")

Valeur Max pour LISTING_RECORD_CERTIFIED_THROUGH: 2021-12-31 00:00:00
Valeur Min pour LISTING_RECORD_CERTIFIED_THROUGH: 2020-12-31 00:00:00


### Attributs présent uniquement dans la table Package:

#### NDCPACKAGECODE

Cet attribut prend la valeur du `PRODUCTNDC` + un code correspondant au package, sous le format 4-4-2, 5-3-2, ou 5-4-1.
Des valeurs sont manquantes et d'autres aberrantes, comme la présence de dates.

In [47]:
# Show an outlier with a date as NDCPACKAGECODE
package["NDCPACKAGECODE"][41901]

'20210901'

#### PACKAGEDESCRIPTION

Ceci est une zone de description contenant un duplicata du `NDCPACKAGECODE` et décrit la quantité de médicaments
dans chaque package et sous-package. On peut aussi noter qu'aucune donnée n'est manquante, il nous sera donc possible
d'utiliser cette colonne pour récupérer le NDC si nécessaire.

#### SAMPLE_PACKAGE

Cet attribut présente un `Y` quand un package est à distribuer en tant qu'échantillon.
Il n'y a pas de valeur manquante, et les données sont correctes.

## 1.6 Résumé

 
Beaucoup de cases sont vides cependant, il nous semble difficile de retrouver la plupart des valeurs sans l'aide d'un expert.  
Certains attributs ont des valeurs qui manquent de cohérence, notamment au niveau des dates.
Pour certaines données, on ne sait pas si elles sont à jour, notre hypothèse est qu'elles le sont.  
Un mélange de bruits et de données aberrantes est présent dans l'ensemble des 2 tables.


## 3. Détectez et corrigez les incohérences entre des valeurs d’attributs dans les deux tables

In [5]:
### Checking NDC related columns

# def regex for finding NDC
NDCRegex = r"\d{4,5}-\d{4}|\d{5}-\d{3}"

# def function to extract the NDC from a col
def NDCGetter(row, baseCol):
    searchResult = re.search(NDCRegex, str(row[baseCol]))
    if searchResult:
        return searchResult.group()
    return np.NaN

# def function to replace the incoherent values of NDC by the extracted ones of the base column
def copyNDC(df, baseCol, destCol):
    validNDC = df.apply(NDCGetter, axis=1, baseCol=baseCol)
    invalidNDCMask = (df[destCol] != validNDC) & ~df[baseCol].isna()
    
    if invalidNDCMask.sum() > 0:
        df[destCol].mask(invalidNDCMask, validNDC, inplace=True)
        print(f"{invalidNDCMask.sum()} incohérence(s) ont été réparée(s) entre {destCol} et {baseCol}.")
    else:
        print(f"Il n'y a aucune incohérence entre {destCol} et {baseCol}.")

## Checking package table
# Checking coherency of PRODUCTID and PACKAGEDESCRIPTION
packageIdNDC = package.apply(NDCGetter, axis=1, baseCol="PRODUCTID")
descNDC = package.apply(NDCGetter, axis=1, baseCol="PACKAGEDESCRIPTION")
packageIncoherence = packageIdNDC != descNDC

if packageIncoherence.sum() > 0:
    print(f"Il y a {packageIncoherence.sum()} incohérence(s) entre PRODUCTID et PACKAGEDESCRIPTION dans package :")
    print(package[["PRODUCTID", "PACKAGEDESCRIPTION"]][packageIncoherence])
else:
    print("Il n'y a pas d'incohérence entre PRODUCTID et PACKAGEDESCRIPTION dans package.")
    
## Checking product table
# Checking coherency of PRODUCTID and PRODUCTNDC
productIdNDC = product.apply(NDCGetter, axis=1, baseCol="PRODUCTID")
productNDC = product.apply(NDCGetter, axis=1, baseCol="PRODUCTNDC")
productIncoherence = (productIdNDC != productNDC) #& ~product["PRODUCTID"].isna() # just show incoherence not due to missing values

if productIncoherence.sum() > 0:
    print(f"Il y a {productIncoherence.sum()} incohérence(s) entre PRODUCTID et PRODUCTNDC dans product :")
    print(product[["PRODUCTID", "PRODUCTNDC"]][productIncoherence])
else:
    print("Il n'y a pas d'incohérence entre PRODUCTID et PRODUCTNDC dans product.")

Il y a 1 incohérence(s) entre PRODUCTID et PACKAGEDESCRIPTION dans package :
                                           PRODUCTID  \
6913  0220-2997_8590fcc9-c0c3-f43b-e053-2991aa0a692e   

                      PACKAGEDESCRIPTION  
6913  1 [hp_M] in 1 TUBE (0220-1997-41)   
Il y a 2073 incohérence(s) entre PRODUCTID et PRODUCTNDC dans product :
                                            PRODUCTID           PRODUCTNDC
0                                                 NaN            0002-0800
1                                                 NaN            0002-1200
2                                                 NaN            0002-1433
3                                                 NaN            0002-1434
4                                                 NaN            0002-1436
...                                               ...                  ...
44665  54111-121_6e1d8cfd-2d32-41f8-a9db-73501fadc90c  OTC MONOGRAPH FINAL
44666  54111-122_f49399e7-0fe0-4db1-baf1-47a7241066b

#### Table `package`

Les colonnes `PRODUCTID` et `PACKAGEDESCRIPTION` sont cohérentes entre-elles excépté pour le produit `6913`.
 En utilisant le [site de recherche de la FDA](https://www.accessdata.fda.gov/scripts/cder/ndc/index.cfm) on peut
 remarquer que le NDC retenu pour ce produit est celui pouvant être trouvé dans la description (soit : `0220-1997`)
 
Par conséquent, nous allons donc maintenant considérer que les NDC trouvé dans la description sont les bons et les
utiliser pour réparer les incohérences.

#### Table `product`

On peut noter qu'il existe beaucoup d'incohérence entre `PRODUCTID` et `PRODUCTNDC` dû à la présence de bruits et de
données abérentes dans `PRODUCTNDC` mais aussi à cause des données manquantes de `PRODUCTID`. Pour retrouver les IDs
manquant, nous utiliserons la table package afin de retrouver les hashes des "SPL document ID".

In [6]:
### Repairing package NDC
# Replace incoherent PRODUCTID in package
getDocId = lambda row: re.search(r"_.+$", row).group()
package["PRODUCTID"].mask(packageIncoherence, descNDC + package["PRODUCTID"].apply(getDocId), inplace=True)
# Replace incoherent PRODUCTNDC in package
copyNDC(package, "PACKAGEDESCRIPTION", "PRODUCTNDC")

### Repairing product NDC
# Replace incoherent PRODUCTNDC in product
copyNDC(product, "PRODUCTID", "PRODUCTNDC")

# Replace missing PRODUCTID in product
def getPackageDocIdFromNDC(NDC):
    rowId = package[package["PRODUCTNDC"] == NDC].index
    if rowId.empty:
        return np.NaN
    return package["PRODUCTID"][rowId[0]]

missingId = productNDC[product["PRODUCTID"].isna()].apply(getPackageDocIdFromNDC)
product["PRODUCTID"].mask(product["PRODUCTID"].isna(), missingId, inplace=True)

2020 incohérence(s) ont été réparée(s) entre PRODUCTNDC et PACKAGEDESCRIPTION.
513 incohérence(s) ont été réparée(s) entre PRODUCTNDC et PRODUCTID.


In [51]:
### Checking STARTMARKETINGDATE is before ENDMARKETINGDATE

# STARTMARKETINGDATE must be before ENDMARKETINGDATE
productInvalidDateMask = product["STARTMARKETINGDATE"] > product["ENDMARKETINGDATE"]
packageInvalidDateMask = package["STARTMARKETINGDATE"] > package["ENDMARKETINGDATE"]

if productInvalidDateMask.sum() > 0:
    print(f"Il y a {productInvalidDateMask.sum()} incohérence(s) entre STARTMARKETINGDATE et ENDMARKETINGDATE dans product.")
else:
    print("Il n'y a pas d'incohérence entre STARTMARKETINGDATE et ENDMARKETINGDATE dans product.")
if packageInvalidDateMask.sum() > 0:
    print(f"Il y a {packageInvalidDateMask.sum()} incohérence(s) entre STARTMARKETINGDATE et ENDMARKETINGDATE dans package.")
else:
    print("Il n'y a pas d'incohérence entre STARTMARKETINGDATE et ENDMARKETINGDATE dans package.")

Il n'y a pas d'incohérence entre STARTMARKETINGDATE et ENDMARKETINGDATE dans product.
Il n'y a pas d'incohérence entre STARTMARKETINGDATE et ENDMARKETINGDATE dans package.


In [52]:
### Check the LISTING_RECORD_CERTIFIED_THROUGH values in product

# the date should be later than now to be considered up to date
expiredRecords = product["LISTING_RECORD_CERTIFIED_THROUGH"] < datetime.now()

if expiredRecords.sum() > 0:
    print(f"Il y a {expiredRecords.sum()} donnée(s) marquée(s) comme expirée(s) dans product.")
else:
    print("Il n'y a aucune donnée marquée comme expirée dans product.")

Il n'y a aucune donnée marquée comme expirée dans product.


In [53]:
### Checking NDC_EXCLUDE_FLAG

productInvalidExcludeFlag = product["NDC_EXCLUDE_FLAG"] != 'N'
packageInvalidExcludeFlag = package["NDC_EXCLUDE_FLAG"] != 'N'

if productInvalidExcludeFlag.sum() > 0:
    print(f"Il y a {productInvalidExcludeFlag.sum()} marquée(s) comme dépassée(s) (non à jour) dans product.")
else:
    print("Il n'y a aucune donnée marquée comme expirée dans product.")
if packageInvalidExcludeFlag.sum() > 0:
    print(f"Il y a {packageInvalidExcludeFlag.sum()} marquée(s) comme dépassée(s) (non à jour) dans package.")
else:
    print("Il n'y a aucune donnée marquée comme expirée dans package.")

Il n'y a aucune donnée marquée comme expirée dans product.
Il n'y a aucune donnée marquée comme expirée dans package.


In [10]:
# Define a generic function for checking the rightness of a column's value according to a dict and allowing replacement
def checkColumnFromDict(row, columnName=None, AcceptableTerms=None):    
    # check param
    if columnName is None or AcceptableTerms is None:
        return np.NaN
    if row[columnName] is np.NaN:
        return np.NaN
    
    # split for multiple value columns
    values = [val.strip() for val in row[columnName].split(';')]
    
    for i, value in enumerate(values):
        if isinstance(AcceptableTerms, dict):
            values[i] = AcceptableTerms.get(value, np.NaN)
        elif not value in AcceptableTerms:
            values[i] = np.NaN
    
    if np.NaN in values:
        return np.NaN
    return '; '.join(values)    # we don't separate the result into multiple columns yet

In [11]:
### Check the DOSAGEFORMNAME values in product

# list found from https://www.fda.gov/industry/structured-product-labeling-resources/dosage-forms
# the different sub-dosage forms are removed to simplify the column values
DosageSPLAcceptableTerm = {"AEROSOL": "AEROSOL","AEROSOL, FOAM": "AEROSOL","AEROSOL, METERED": "AEROSOL","AEROSOL, POWDER": "AEROSOL","AEROSOL, SPRAY": "AEROSOL","BAR": "BAR","BAR, CHEWABLE": "BAR","BEAD": "BEAD","CAPSULE": "CAPSULE","CAPSULE, COATED": "CAPSULE","CAPSULE, COATED PELLETS": "CAPSULE","CAPSULE, COATED, EXTENDED RELEASE": "CAPSULE","CAPSULE, DELAYED RELEASE": "CAPSULE","CAPSULE, DELAYED RELEASE PELLETS": "CAPSULE","CAPSULE, EXTENDED RELEASE": "CAPSULE","CAPSULE, FILM COATED, EXTENDED RELEASE": "CAPSULE","CAPSULE, GELATIN COATED": "CAPSULE","CAPSULE, LIQUID FILLED": "CAPSULE","CELLULAR SHEET": "CELLULAR SHEET","CHEWABLE GEL": "CHEWABLE GEL","CLOTH": "CLOTH","CONCENTRATE": "CONCENTRATE","CREAM": "CREAM","CREAM, AUGMENTED": "CREAM","CRYSTAL": "CRYSTAL","DISC": "DISC","DOUCHE": "DOUCHE","DRESSING": "DRESSING","ELIXIR": "ELIXIR","EMULSION": "EMULSION","ENEMA": "ENEMA","EXTRACT": "EXTRACT","FIBER, EXTENDED RELEASE": "FIBER","FILM": "FILM","FILM, EXTENDED RELEASE": "FILM","FILM, SOLUBLE": "FILM","FOR SOLUTION": "FOR SOLUTION","FOR SUSPENSION": "FOR SUSPENSION","FOR SUSPENSION, EXTENDED RELEASE": "FOR SUSPENSION","GAS": "GAS","GEL": "GEL","GEL, DENTIFRICE": "GEL","GEL, METERED": "GEL","GLOBULE": "GLOBULE","GRANULE": "GRANULE","GRANULE, DELAYED RELEASE": "GRANULE","GRANULE, EFFERVESCENT": "GRANULE","GRANULE, FOR SOLUTION": "GRANULE","GRANULE, FOR SUSPENSION": "GRANULE","GRANULE, FOR SUSPENSION, EXTENDED RELEASE": "GRANULE","GUM": "GUM","GUM, CHEWING": "GUM","IMPLANT": "IMPLANT","INHALANT": "INHALANT","INJECTABLE FOAM": "INJECTABLE FOAM","INJECTABLE": "INJECTABLE","INJECTABLE, LIPOSOMAL": "INJECTABLE","INJECTION": "INJECTION","INJECTION, EMULSION": "INJECTION","INJECTION, LIPID COMPLEX": "INJECTION","INJECTION, POWDER, FOR SOLUTION": "INJECTION","INJECTION, POWDER, FOR SUSPENSION": "INJECTION","INJECTION, POWDER, FOR SUSPENSION, EXTENDED RELEASE": "INJECTION","INJECTION, POWDER, LYOPHILIZED, FOR LIPOSOMAL SUSPENSION": "INJECTION","INJECTION, POWDER, LYOPHILIZED, FOR SOLUTION": "INJECTION","INJECTION, POWDER, LYOPHILIZED, FOR SUSPENSION": "INJECTION","INJECTION, POWDER, LYOPHILIZED, FOR SUSPENSION, EXTENDED RELEASE": "INJECTION","INJECTION, SOLUTION": "INJECTION","INJECTION, SOLUTION, CONCENTRATE": "INJECTION","INJECTION, SUSPENSION": "INJECTION","INJECTION, SUSPENSION, EXTENDED RELEASE": "INJECTION","INJECTION, SUSPENSION, LIPOSOMAL": "INJECTION","INJECTION, SUSPENSION, SONICATED": "INJECTION","INSERT": "INSERT","INSERT, EXTENDED RELEASE": "INSERT","INTRAUTERINE DEVICE": "INTRAUTERINE DEVICE","IRRIGANT": "IRRIGANT","JELLY": "JELLY","KIT": "KIT","LINIMENT": "LINIMENT","LIPSTICK": "LIPSTICK","LIQUID": "LIQUID","LIQUID, EXTENDED RELEASE": "LIQUID","LOTION": "LOTION","LOTION, AUGMENTED": "LOTION","LOTION/SHAMPOO": "LOTION/SHAMPOO","LOZENGE": "LOZENGE","MOUTHWASH": "MOUTHWASH","NOT APPLICABLE": "NOT APPLICABLE","OIL": "OIL","OINTMENT": "OINTMENT","OINTMENT, AUGMENTED": "OINTMENT","PASTE": "PASTE","PASTE, DENTIFRICE": "PASTE","PASTILLE": "PASTILLE","PATCH": "PATCH","PATCH, EXTENDED RELEASE": "PATCH","PATCH, EXTENDED RELEASE, ELECTRICALLY CONTROLLED": "PATCH","PELLET": "PELLET","PELLET, IMPLANTABLE": "PELLET","PELLETS, COATED, EXTENDED RELEASE": "PELLETS","PILL": "PILL","PLASTER": "PLASTER","POULTICE": "POULTICE","POWDER": "POWDER","POWDER, DENTIFRICE": "POWDER","POWDER, FOR SOLUTION": "POWDER","POWDER, FOR SUSPENSION": "POWDER","POWDER, METERED": "POWDER","RING": "RING","RINSE": "RINSE","SALVE": "SALVE","SHAMPOO": "SHAMPOO","SHAMPOO, SUSPENSION": "SHAMPOO","SOAP": "SOAP","SOLUTION": "SOLUTION","SOLUTION, CONCENTRATE": "SOLUTION","SOLUTION, FOR SLUSH": "SOLUTION","SOLUTION, GEL FORMING / DROPS": "SOLUTION","SOLUTION, GEL FORMING, EXTENDED RELEASE": "SOLUTION","SOLUTION/ DROPS": "SOLUTION/ DROPS","SPONGE": "SPONGE","SPRAY": "SPRAY","SPRAY, METERED": "SPRAY","SPRAY, SUSPENSION": "SPRAY","STICK": "STICK","STRIP": "STRIP","SUPPOSITORY": "SUPPOSITORY","SUPPOSITORY, EXTENDED RELEASE": "SUPPOSITORY","SUSPENSION": "SUSPENSION","SUSPENSION, EXTENDED RELEASE": "SUSPENSION","SUSPENSION/ DROPS": "SUSPENSION/ DROPS","SWAB": "SWAB","SYRUP": "SYRUP","SYSTEM": "SYSTEM","TABLET": "TABLET","TABLET, CHEWABLE": "TABLET","TABLET, CHEWABLE, EXTENDED RELEASE": "TABLET","TABLET, COATED": "TABLET","TABLET, COATED PARTICLES": "TABLET","TABLET, DELAYED RELEASE": "TABLET","TABLET, DELAYED RELEASE PARTICLES": "TABLET","TABLET, EFFERVESCENT": "TABLET","TABLET, EXTENDED RELEASE": "TABLET","TABLET, FILM COATED": "TABLET","TABLET, FILM COATED, EXTENDED RELEASE": "TABLET","TABLET, FOR SOLUTION": "TABLET","TABLET, FOR SUSPENSION": "TABLET","TABLET, MULTILAYER": "TABLET","TABLET, MULTILAYER, EXTENDED RELEASE": "TABLET","TABLET, ORALLY DISINTEGRATING": "TABLET","TABLET, ORALLY DISINTEGRATING, DELAYED RELEASE": "TABLET","TABLET, SOLUBLE": "TABLET","TABLET, SUGAR COATED": "TABLET","TABLET WITH SENSOR": "TABLET WITH SENSOR","TAMPON": "TAMPON","TAPE": "TAPE","TINCTURE": "TINCTURE","TROCHE": "TROCHE","WAFER": "WAFER"}

product["DOSAGEFORMNAME"] = product.apply(checkColumnFromDict, axis=1, columnName="DOSAGEFORMNAME", AcceptableTerms=DosageSPLAcceptableTerm)
nbNotAccepted = product["DOSAGEFORMNAME"].isna().sum()

if nbNotAccepted > 0:
    print(f"Il y a {nbNotAccepted} forme(s) de dosage(s) invalide(s).")
else:
    print("Il n'y a aucune forme de dosage invalide.")

Il n'y a aucune forme de dosage invalide.


In [12]:
### Check the DOSAGEFORMNAME values in product

# list found from https://www.fda.gov/industry/structured-product-labeling-resources/route-administration
routeSPLAcceptableTerm = ["AURICULAR (OTIC)","BUCCAL","CONJUNCTIVAL","CUTANEOUS","DENTAL","ELECTRO-OSMOSIS","ENDOCERVICAL","ENDOSINUSIAL","ENDOTRACHEAL","ENTERAL","EPIDURAL","EXTRA-AMNIOTIC","EXTRACORPOREAL","HEMODIALYSIS","INFILTRATION","INTERSTITIAL","INTRA-ABDOMINAL","INTRA-AMNIOTIC","INTRA-ARTERIAL","INTRA-ARTICULAR","INTRABILIARY","INTRABRONCHIAL","INTRABURSAL","INTRACANALICULAR","INTRACARDIAC","INTRACARTILAGINOUS","INTRACAUDAL","INTRACAVERNOUS","INTRACAVITARY","INTRACEREBRAL","INTRACISTERNAL","INTRACORNEAL","INTRACORONAL, DENTAL","INTRACORONARY","INTRACORPORUS CAVERNOSUM","INTRACRANIAL","INTRADERMAL","INTRADISCAL","INTRADUCTAL","INTRADUODENAL","INTRADURAL","INTRAEPICARDIAL","INTRAEPIDERMAL","INTRAESOPHAGEAL","INTRAGASTRIC","INTRAGINGIVAL","INTRAHEPATIC","INTRAILEAL","INTRALESIONAL","INTRALINGUAL","INTRALUMINAL","INTRALYMPHATIC","INTRAMAMMARY","INTRAMEDULLARY","INTRAMENINGEAL","INTRAMUSCULAR","INTRANODAL","INTRAOCULAR","INTRAOMENTUM","INTRAOVARIAN","INTRAPERICARDIAL","INTRAPERITONEAL","INTRAPLEURAL","INTRAPROSTATIC","INTRAPULMONARY","INTRARUMINAL","INTRASINAL","INTRASPINAL","INTRASYNOVIAL","INTRATENDINOUS","INTRATESTICULAR","INTRATHECAL","INTRATHORACIC","INTRATUBULAR","INTRATUMOR","INTRATYMPANIC","INTRAUTERINE","INTRAVASCULAR","INTRAVENOUS","INTRAVENTRICULAR","INTRAVESICAL","INTRAVITREAL","IONTOPHORESIS","IRRIGATION","LARYNGEAL","NASAL","NASOGASTRIC","NOT APPLICABLE","OCCLUSIVE DRESSING TECHNIQUE","OPHTHALMIC","ORAL","OROPHARYNGEAL","PARENTERAL","PERCUTANEOUS","PERIARTICULAR","PERIDURAL","PERINEURAL","PERIODONTAL","RECTAL","RESPIRATORY (INHALATION)","RETROBULBAR","SOFT TISSUE","SUBARACHNOID","SUBCONJUNCTIVAL","SUBCUTANEOUS","SUBGINGIVAL","SUBLINGUAL","SUBMUCOSAL","SUBRETINAL","TOPICAL","TRANSDERMAL","TRANSENDOCARDIAL","TRANSMUCOSAL","TRANSPLACENTAL","TRANSTRACHEAL","TRANSTYMPANIC","URETERAL","URETHRAL","VAGINAL"]

product["ROUTENAME"] = product.apply(checkColumnFromDict, axis=1, columnName="ROUTENAME", AcceptableTerms=routeSPLAcceptableTerm)
nbNotAccepted = product["ROUTENAME"].isna().sum() - 1932 # remove the number of initially missing values

if nbNotAccepted > 0:
    print(f"Il y a {nbNotAccepted} voie(s) d'administration(s) invalide(s).")
else:
    print("Il n'y a aucune voie d'administration invalide.")

Il n'y a aucune voie d'administration invalide.


In [13]:
### Check the MARKETINGCATEGORYNAME values in product

# list found from https://www.fda.gov/industry/structured-product-labeling-resources/marketing-category
marketCatSPLAcceptableTerm = {"NDA, ANDA, or BLA": "NDA, ANDA, or BLA", "NDA": "NDA, ANDA, or BLA", "ANDA": "NDA, ANDA, or BLA", "BLA": "NDA, ANDA, or BLA", "NDA AUTHORIZED GENERIC": "NDA, ANDA, or BLA", "OTC": "OTC", "OTC MONOGRAPH FINAL": "OTC", "OTC MONOGRAPH NOT FINAL" : "OTC", "UNAPPROVED": "UNAPPROVED", "UNAPPROVED DRUG OTHER": "UNAPPROVED", "UNAPPROVED HOMEOPATHIC": "UNAPPROVED", "UNAPPROVED MEDICAL GAS": "UNAPPROVED", "UNAPPROVED DRUG FOR USE IN DRUG SHORTAGE": "UNAPPROVED"}

product["MARKETINGCATEGORYNAME"] = product.apply(checkColumnFromDict, axis=1, columnName="MARKETINGCATEGORYNAME", AcceptableTerms=marketCatSPLAcceptableTerm)
nbNotAccepted = product["MARKETINGCATEGORYNAME"].isna().sum()

if nbNotAccepted > 0:
    print(f"Il y a {nbNotAccepted} catégorie(s) de marketing invalide(s).")
else:
    print("Il n'y a aucune catégorie de marketing invalide.")

Il n'y a aucune catégorie de marketing invalide.


In [14]:
### Check the DEASCHEDULE values in product

# list found from https://www.fda.gov/industry/structured-product-labeling-resources/spl-dea-schedule
DEASPLAcceptableTerm = ["CI","CII","CIII","CIV","CV"]

product["DEASCHEDULE"] = product.apply(checkColumnFromDict, axis=1, columnName="DEASCHEDULE", AcceptableTerms=DEASPLAcceptableTerm)
nbNotAccepted = product["DEASCHEDULE"].isna().sum() - 88815 # remove the number of initially missing values

if nbNotAccepted > 0:
    print(f"Il y a {nbNotAccepted} donnée(s) invalide(s) dans DEASCHEDULE.")
else:
    print("Il n'y a aucune donnée invalide dans DEASCHEDULE.")

Il n'y a aucune donnée invalide dans DEASCHEDULE.


In [15]:
### Check the SAMPLE_PACKAGE values in package

sampleSPLAcceptableTerm = ['Y','N']

package["SAMPLE_PACKAGE"] = package.apply(checkColumnFromDict, axis=1, columnName="SAMPLE_PACKAGE", AcceptableTerms=sampleSPLAcceptableTerm)
nbNotAccepted = package["SAMPLE_PACKAGE"].isna().sum() # remove the number of initially missing values

if nbNotAccepted > 0:
    print(f"Il y a {nbNotAccepted} donnée(s) invalide(s) dans SAMPLE_PACKAGE.")
else:
    print("Il n'y a aucune donnée invalide dans SAMPLE_PACKAGE.")

Il n'y a aucune donnée invalide dans SAMPLE_PACKAGE.


In [16]:
### Check the APPLICATIONNUMBER values in product using MARKETINGCATEGORYNAME

appNbRegex = {"NDA, ANDA, or BLA": r"^(NDA|BLA|BN|BA|ANDA)\d+$", "OTC": r"^(part\w+,?)+$", "UNAPPROVED": r''}
appNbChecker = lambda row: bool(re.match(appNbRegex[row["MARKETINGCATEGORYNAME"]], str(row["APPLICATIONNUMBER"])))
validAppNb = product.apply(appNbChecker, axis=1)

product[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]][~validAppNb]

,MARKETINGCATEGORYNAME,APPLICATIONNUMBER
90014,OTC,NaN


On voit sur ces résultats que le produit 26428 et 90014 possèdent un numéro d'application erronés.

Pour le 26428, on peut facilement supposer que le terme `part` a été oublié.
Cependant, pour le 90014, il nous est difficile de retrouver le numéro de la citation CFR.

In [18]:
if not (product.at[26428, "APPLICATIONNUMBER"].startswith("part") and product.at[90014, "APPLICATIONNUMBER"] is np.NaN):
    product.at[26428, "APPLICATIONNUMBER"] = "part" + product["APPLICATIONNUMBER"][26428]
    product.at[90014, "APPLICATIONNUMBER"] = np.NaN
    print("Le numéro d'application du produit 26428 à été réparé et celui du produit 90014 supprimé.")
else:
    print("Le numéro d'application du produit 26428 à déjà été réparé et celui du produit 90014 est déjà supprimé.")

product[["MARKETINGCATEGORYNAME","APPLICATIONNUMBER"]][~validAppNb]

Le numéro d'application du produit 26428 à déjà été réparé et celui du produit 90014 est déjà supprimé.


,MARKETINGCATEGORYNAME,APPLICATIONNUMBER
90014,OTC,NaN


In [19]:
### Check the number of values in SUBSTANCENAME, ACTIVE_NUMERATOR_STRENGTH and ACTIVE_INGRED_UNIT

# their is a product with a substance composed of two substance ("GLYCERIN; HYDROLYZED SOY PROTEIN (ENZYMATIC, 2000 MW)")
# so we must remove the ';' inside the parenthesis before splitting
multiValueCounter = lambda row, col: len(re.sub(r"(\().*?(\))", '', row[col]).split(';')) if isinstance(row[col], str) else 0
subColsLengthChecker = lambda row: multiValueCounter(row, "SUBSTANCENAME") == multiValueCounter(row, "ACTIVE_NUMERATOR_STRENGTH") == multiValueCounter(row, "ACTIVE_INGRED_UNIT")

invalidRows = ~product.apply(subColsLengthChecker, axis=1)

if invalidRows.sum() > 0:
    print("Substances des produit invalides :")
    for i in product.index[invalidRows]:
        print(product["SUBSTANCENAME"][i])
else:
    print("Il n'y a aucune incohérence dans le nombre de valeurs des colonnes SUBSTANCENAME, ACTIVE_NUMERATOR_STRENGTH et ACTIVE_INGRED_UNIT")

Substances des produit invalides :
INFLUENZA A VIRUS A/IDAHO/07/2018 (H1N1) ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED; INFLUENZA A VIRUS A/INDIANA/08/2018 (H3N2) ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA B VIRUS B/SINGAPORE/INFTT-16-0610/2016 ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA B VIRUS B/IOWA/06/2017 ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED)
INFLUENZA A VIRUS A/IDAHO/07/2018 (H1N1) ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED; INFLUENZA A VIRUS A/INDIANA/08/2018 (H3N2) ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA B VIRUS B/SINGAPORE/INFTT-16-0610/2016 ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA B VIRUS B/IOWA/06/2017 ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED)


On peut voir sur les deux produits invalides qu'il y a une coquille : une parenthèse n'est pas fermée.

In [20]:
if product.at[81044, "SUBSTANCENAME"].count('(') > product.at[81044, "SUBSTANCENAME"].count(')'):
    for i in product.index[invalidRows]:
        product.at[i, "SUBSTANCENAME"] = product.at[i, "SUBSTANCENAME"][:94] + ')' + product.at[i, "SUBSTANCENAME"][94:]
else:
    print("Les substances des produits 81044 et 81045 ont déjà été réparés.")
   
print("\nSubstances des produit corrigés :")
for i in product.index[invalidRows]:
    print(product["SUBSTANCENAME"][i])


Substances des produit corrigés :
INFLUENZA A VIRUS A/IDAHO/07/2018 (H1N1) ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA A VIRUS A/INDIANA/08/2018 (H3N2) ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA B VIRUS B/SINGAPORE/INFTT-16-0610/2016 ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA B VIRUS B/IOWA/06/2017 ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED)
INFLUENZA A VIRUS A/IDAHO/07/2018 (H1N1) ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA A VIRUS A/INDIANA/08/2018 (H3N2) ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA B VIRUS B/SINGAPORE/INFTT-16-0610/2016 ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED); INFLUENZA B VIRUS B/IOWA/06/2017 ANTIGEN (MDCK CELL DERIVED, PROPIOLACTONE INACTIVATED)


## 4. Complétez au maximum les données manquantes dans les deux tables

In [50]:
print("Nombre de valeurs manquantes dans product:")
print(product.isna().sum())
print()
print("Nombre de valeurs manquantes dans package:")
print(package.isna().sum())

Nombre de valeurs manquantes dans product:
PRODUCTID                               0
PRODUCTNDC                              0
PRODUCTTYPENAME                         0
PROPRIETARYNAME                         6
PROPRIETARYNAMESUFFIX               83075
NONPROPRIETARYNAME                      4
DOSAGEFORMNAME                          0
ROUTENAME                            1932
STARTMARKETINGDATE                     20
ENDMARKETINGDATE                    88916
MARKETINGCATEGORYNAME                   0
APPLICATIONNUMBER                   13098
LABELERNAME                             0
SUBSTANCENAME                        2309
ACTIVE_NUMERATOR_STRENGTH            2309
ACTIVE_INGRED_UNIT                   2309
PHARM_CLASSES                       50984
DEASCHEDULE                         88815
NDC_EXCLUDE_FLAG                        0
LISTING_RECORD_CERTIFIED_THROUGH     4325
dtype: int64

Nombre de valeurs manquantes dans package:
PRODUCTID                  0
PRODUCTNDC                 0
ND

Il y a donc des données manquantes dans `product` pour les colonnes :
- `PRODUCTID`
- `PROPRIETARYNAME`
- `PROPRIETARYNAMESUFFIX`
- `NONPROPRIETARYNAME`
- `ROUTENAME`
- `ENDMARKETINGDATE`
- `APPLICATIONNUMBER`
- `SUBSTANCENAME`
- `ACTIVE_NUMERATOR_STRENGTH`
- `ACTIVE_INGRED_UNIT`
- `PHARM_CLASSES` 
- `DEASCHEDULE`
- `LISTING_RECORD_CERTIFIED_THROUGH`
    
Il y a donc des données manquantes dans `package` pour les colonnes :
- `PRODUCTNDC`
- `NDCPACKAGECODE`
- `ENDMARKETINGDATE`

Les attributs manquant dont nous ne nous interesserons pas sont :
- `ENDMARKETINGDATE` : Cet attribut est facultatif.
- `DEASCHEDULE` : Les valeurs manquante ont une signification.
- `PHARM_CLASSES` : Nous nous en occuperons plus tard dans la partie 8.
- `LISTING_RECORD_CERTIFIED_THROUGH` : Nous n'avons pas de source sur laquelle nous baser pour retrouver les valeurs manquantes.
- `PROPRIETARYNAMESUFFIX` : Cet attribut est facultatif.
- `PROPRIETARYNAME` : En utilisant les autres attributs (`LABELERNAME` et `NONPROPRIETARYNAME` par exemple) on pourrait
peut-être retrouver les nom au cas pas cas sur internet, mais nous ne jugeont pas que cela soit nécessaire pour ce TP.
- `APPLICATIONNUMBER` : En utilisant les autres attributs (`LABELERNAME` et `NONPROPRIETARYNAME` par exemple) on pourrait
peut-être retrouver les nom au cas pas cas sur internet, mais nous ne jugeont pas que cela soit nécessaire pour ce TP.
- `ACTIVE_NUMERATOR_STRENGTH` et `ACTIVE_INGRED_UNIT` : Il nous serait peut-être possible de retrouver ces valeurs sur
internet, cependant nous ne jugeont pas que cela soit nécessaire pour ce TP.
- `ROUTENAME` : Un lien avec `DOSAGEFORMNAME` existe mais nous pouvons trouver assez vite un contre exemple qui empêche la généralisation.
- `NONPROPRIETARYNAME` : Une certaine cohérence est visible avec `SUBSTANCENAME` mais elle n'est pas parfaite.
- `SUBSTANCENAME` : Une certaine cohérence est visible avec `NONPROPRIETARYNAME` mais elle n'est pas parfaite.

Les attributs manquant qui sont récupérable sont :

`PRODUCTID` : On peut possiblement le récupérer dans l'autre table grâce aux colonnes `PRODUCTNDC`, `NDCPACKAGECODE` et `PACKAGEDESCRIPTION`
`PRODUCTNDC` : On peut le récupérer dans `PRODUCTID`, `NDCPACKAGECODE` et `PACKAGEDESCRIPTION`
`NDCPACKAGECODE` : On peut le récupérer dans `PACKAGEDESCRIPTION`

Nous allons donc utiliser la colonne `PACKAGEDESCRIPTION` pour récupérer les valeurs manquantes dans la table `package` et
la colonne `PRODUCTID` dans la table `product`.

In [51]:
# Done before

20

## 5. Détectez et retirez les objets dupliqués dans les deux tables

## 6. Intégrez les deux tables et nettoyez le résultat

## 7. Proposez un nouvel ensemble d’attributs (représentation) qui élimine la redondance des informations dans les valeurs des attributs

## 8. À partir de la nouvelle représentation, proposez un ensemble d’attributs à utiliser pour prédire toutes les classes pharmaceutiques d’un médicament

## 9. Appliquez un modèle de classification pour prédire les classes pharmaceutiques des médicaments pour lesquels l’information est manquante

## 10. Évaluez un échantillon de vos résultats à l’aide de connaissances d’experts